# Data Preparation

In this workbook I prepare the data for analysis. This job involves

1. Developing an outcome measure - in this case, binary, as there really isn't enough information to do anything else. 
2. Developing a couple of measures of whether there was a surgical intervention, and if so, what kind (intrusive or just an amputation). 
3. Developing some other variables - dummy variables for regiment and injury location, a measure of wound severity based on the wound description, and a "long name" measure, along with a regimental casualties measure. 

First, a little bit of work to get python/Stata in the right working directory:

In [1]:
import ipystata
import os

cd = os.getcwd()
cdl = [cd]

Changing the working directory to the right place - it should work for anyone who has correctly cloned the git repository.

In [2]:
%%stata -s gl -i cdl 
chdir "`cdl'"

clear all 
set more off 

import excel "`cdl'\DataFiles\VicksburgSurgery.xlsx", sheet("Sheet1") firstrow case(lower)

describe


C:\Users\mjbaker\Documents\GitHub\CivilWarSurgeryPublic

Contains data
  obs:           576                          
 vars:            10                          
 size:       202,752                          
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
type            str20   %20s                  Type
case            int     %10.0g                Case
name            str11   %11s                  Name
regno           str3    %9s                   RegNo
regstate        str18   %18s                  RegState
injury          str99   %99s                  Injury
operation       str107  %107s         

# Dealing with missing outcomes 

First, I can't use anything that doesn't report an outcome, so I drop these observations. But..


**Note**: A referee wanted some validation that these observations could plausibly be assumed to be missing at random. We should check and see what the severity of these injuries was. 

In [3]:
%%stata -s gl -os

gen misdum = 0
replace misdum = 1 if remarks == "" | remarks == " " | remarks == "unknown" | remarks == "Unknown"
replace misdum = 1 if strpos(remarks, "unknown") > 0
replace misdum = 1 if strpos(operation, "Unknown") > 0

tab misdum


(75 real changes made)

(2 real changes made)

(1 real change made)

     misdum |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        498       86.46       86.46
          1 |         78       13.54      100.00
------------+-----------------------------------
      Total |        576      100.00



## Making an outcome variable

Now, based on the case remarks, I can posit whether the outcome was good (outcome=1) or bad (outcome=0). While I tried to push this a bit further into a categorical measure, there really isn't enough data or information to do this. 

Here goes:

In [4]:
%%stata -s gl 

replace remarks         = lower(remarks)
replace operation       = lower(operation)

gen     outcome = .
    
replace outcome = 0 if strpos(remarks, "died") > 0

replace outcome = 0 if strpos(remarks, "no improvement") > 0 

replace outcome = 0 if strpos(remarks, "pain") > 0 |                ///
    strpos(remarks, "ulcerated") > 0 | strpos(remarks, "fever") > 0 ///
    | strpos(remarks, "cannot walk") > 0 & outcome == .

replace outcome = 1 if strpos(remarks, "recovered") > 0
    
replace outcome = 1 if strpos(remarks, "returned to duty") > 0 |    ///
    strpos(remarks, "on duty")

replace outcome = 1 if strpos(remarks, "doing very well") > 0
    
replace outcome = 1 if strpos(remarks, "doing well") > 0 &          ///
    strpos(remarks, "not") == 0
    
replace outcome = 1 if strpos(remarks, "feels well") > 0
    
replace outcome = 1 if strpos(remarks, "walks") > 0
    
replace outcome = 1 if strpos(remarks, "doing tolerably well") > 0 | ///
    strpos(remarks, "tolerably well") > 0
    
replace outcome = 1 if strpos(remarks, "nearly well") > 0
    
replace outcome = 1 if (strpos(remarks, "doing better") > 0         ///
    | strpos(remarks, "better")>0) & strpos(remarks, "not") == 0
    
replace outcome=0 if strpos(remarks, "doing badly") > 0 |           ///
    strpos(remarks, "not doing well") > 0 |                     ///
    strpos(remarks, "very weak") > 0                            ///
    | strpos(remarks, "worse") > 0 |                            ///
    strpos(remarks, "very weak") > 0 |                          ///
    strpos(remarks, "swollen") > 0 |                            ///
    strpos(remarks, "paralyzed") > 0 |                           ///
    strpos(remarks, "doing poorly") > 0
    
replace outcome=0 if strpos(remarks, "pain") > 0 |              ///
    strpos(remarks, "ulcerated") > 0 |                          ///
    strpos(remarks, "fever") > 0 |                               ///
    strpos(remarks, "erysipelas") > 0 |                       ///
    strpos(remarks, "cannot walk") > 0 &                        ///
    outcome==.

replace outcome=1 if strpos(remarks, "saved") > 0 & outcome == .
    
replace outcome=1 if strpos(remarks, "doing finely") > 0 & outcome == .

replace outcome=1 if strpos(remarks, ///
    "had erysipelas doing better prospect good") > 0

replace outcome=1 if strpos(remarks, ///
    "had erysipelas, -better doing well 15th day") > 0 

replace outcome=1 if strpos(remarks, ///
    "had erysipelas on face, doing well 16th day") > 0

replace outcome=1 if strpos(remarks, ///
    "erysipelas, doing better on 17th day") > 0

replace outcome=1 if strpos(remarks, "erysipelas, doing well 18th") > 0

replace outcome=1 if strpos(remarks, ///
    "had erysipelas; pusburrowed better 16th day") > 0

replace outcome=1 if strpos(remarks, "alive") > 0

replace outcome=1 if strpos(remarks, "wearing shoes") > 0

replace outcome=1 if strpos(remarks, "doing passably well") > 0

replace outcome=1 if remarks == "well on 16th day"

replace outcome=1 if strpos(remarks, "flap sloughed") > 0

replace outcome = 0 if strpos(remarks, "no better 16th day") > 0

replace outcome = 0 if strpos(remarks, "very sore 16th day") > 0

replace outcome = 0 if strpos(remarks, "tendency to gangrene") > 0

replace outcome = 0 if strpos(remarks, "considerable swelling") > 0

(502 real changes made)

(575 real changes made)

(576 missing values generated)

(71 real changes made)

(1 real change made)

(7 real changes made)

(56 real changes made)

(10 real changes made)

(4 real changes made)

(290 real changes made)

(1 real change made)

(2 real changes made)

(9 real changes made)

(7 real changes made)

(7 real changes made)

(18 real changes made)

(15 real changes made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(2 real changes made)

(1 real change made)

(1 real change made)

(3 real changes made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)

(1 real change made)



In [5]:
%%stata -s gl

tab outcome


    outcome |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        109       21.84       21.84
          1 |        390       78.16      100.00
------------+-----------------------------------
      Total |        499      100.00



## Operation indicator

Here, we make a dummy variable for a treatment or intervention. This measure does not distinguish about intervention type, but instead is just for whether or not some sort of surgical intervention happend. 

In [6]:
%%stata -s gl -os

gen operated     = 1
replace operated = 0 if strpos(operation, "none") 
replace operated = 0 if strpos(operation, "not")
replace operated = 0 if strpos(operation, "no operation")
replace operated = 0 if operation == ""


(388 real changes made)

(12 real changes made)

(2 real changes made)

(1 real change made)



# Missing variables and the incidence of operations

Tabulating the operation variable and the missing variables seems to indicate that the frequency of operation in each groups was about the same. This seems to support the contention that these observations are missing at random. 

In [7]:
%%stata -s gl
tab operated misdum


           |        misdum
  operated |         0          1 |     Total
-----------+----------------------+----------
         0 |       349         54 |       403 
         1 |       149         24 |       173 
-----------+----------------------+----------
     Total |       498         78 |       576 



# A more expansive intervention measure

A referee correctly argued that a bit more could be done with the intervention measure; specifically, an amputation is very different than a procedure that involves some sort of more surgical intervention, such as a resection, excision, or bullet removal. 

In fact, a lot of the Civil War data makes a distinction between no treatment (conservation), amputation, and resection, so we do our best to mirror this breakdown with the following variable:

In [8]:
%%stata -s gl

generate expoper = operated
replace expoper = 2 if strpos(operation, "ball cut out")     > 0
replace expoper = 2 if strpos(operation, "ball extracted")   > 0 
replace expoper = 2 if strpos(operation, "ball pried loose") > 0
replace expoper = 2 if strpos(operation, "cut out")          > 0
replace expoper = 2 if strpos(operation, "extracted")        > 0 
replace expoper = 2 if strpos(operation, "excision")         > 0 
replace expoper = 2 if strpos(operation, "enlarged")         > 0 
replace expoper = 2 if strpos(operation, "exsection")        > 0 
replace expoper = 2 if strpos(operation, "ligation")         > 0 
replace expoper = 2 if strpos(operation, "sewed up")         > 0
replace expoper = 2 if strpos(operation, "resected")         > 0
replace expoper = 2 if strpos(operation, "resection")        > 0
replace expoper = 2 if strpos(operation, "trepanned")        > 0
replace expoper = 2 if strpos(operation, "cut down and tied") > 0
replace expoper = 2 if strpos(operation, "ligated")           > 0
replace expoper = 2 if strpos(operation, "excised")           > 0
replace expoper = 2 if strpos(operation, "artery tied")       > 0
replace expoper = 2 if strpos(operation, "long splint")       > 0
replace expoper = 0 if strpos(operation, "ball not extracted") > 0
replace expoper = 0 if strpos(operation, "bullet not extracted") >0
replace expoper = 0 if strpos(operation, "not extracted")     > 0
replace expoper = 2 if strpos(operation, "tapped")             > 0
replace expoper = 2 if strpos(operation, "tied the vessel")   > 0



(6 real changes made)

(13 real changes made)

(1 real change made)

(9 real changes made)

(34 real changes made)

(2 real changes made)

(1 real change made)

(1 real change made)

(0 real changes made)

(1 real change made)

(5 real changes made)

(8 real changes made)

(1 real change made)

(1 real change made)

(2 real changes made)

(2 real changes made)

(1 real change made)

(1 real change made)

(7 real changes made)

(3 real changes made)

(1 real change made)

(1 real change made)

(1 real change made)



## Categorical injury variables

Of course, the location of a wound matters a lot for both the application of surgery and survival. Accordingly, we form dummy variables for wound categories: 

In [9]:
%%stata -s gl

replace type   = lower(type)
replace injury = lower(injury)
tab type, gen(td)

(576 real changes made)

(575 real changes made)

                Type |      Freq.     Percent        Cum.
---------------------+-----------------------------------
                 arm |         50        8.68        8.68
               elbow |         14        2.43       11.11
                foot |         45        7.81       18.92
             forearm |         35        6.08       25.00
                hand |         57        9.90       34.90
                head |         39        6.77       41.67
                 hip |         34        5.90       47.57
                knee |         20        3.47       51.04
                 leg |         67       11.63       62.67
neck, trunk shoulder |        127       22.05       84.72
               thigh |         88       15.28      100.00
---------------------+-----------------------------------
               Total |        576      100.00



## Checking on the missing variables

Once again, the missing don't seem to be all that different in terms of wound location.

In [10]:
%%stata -s gl
tab type misdum


                     |        misdum
                Type |         0          1 |     Total
---------------------+----------------------+----------
                 arm |        40         10 |        50 
               elbow |        12          2 |        14 
                foot |        43          2 |        45 
             forearm |        30          5 |        35 
                hand |        46         11 |        57 
                head |        33          6 |        39 
                 hip |        29          5 |        34 
                knee |        18          2 |        20 
                 leg |        64          3 |        67 
neck, trunk shoulder |       101         26 |       127 
               thigh |        82          6 |        88 
---------------------+----------------------+----------
               Total |       498         78 |       576 



## Severity of injury scale

We now mine the "injury" category of Andrews' data, which gives us some clues as to how severe the injury was. Using words that indicate the wound was bad, we form a "severity" scale. In regressions, we can use a polynomial function of the severity scale so as to capture its potentially nonlinear impact on outcomes. 

One can look at the code to see what sorts of things lead to a wound being considered more severe. Also, the very last line of code below generates a squared term so severity can be included in empirical work as a quadratic.

In [11]:
%%stata -s gl
gen sev = 0

replace sev = sev + 2 if strpos(injury, "severe")       > 0
replace sev = sev + 1 if strpos(injury, "entered")      > 0
replace sev = sev + 1 if strpos(injury, "compound")     > 0
replace sev = sev + 1 if strpos(injury, "fracture")     > 0
replace sev = sev + 1 if strpos(injury, "shattered")    > 0
replace sev = sev + 1 if strpos(injury, "penetrat")     > 0
replace sev = sev + 1 if strpos(injury, "lung")         > 0
replace sev = sev + 1 if strpos(injury, "spine")        > 0
replace sev = sev + 1 if strpos(injury, "bad")          > 0
replace sev = sev + 1 if strpos(injury, "taken off")    > 0
replace sev = sev + 1 if strpos(injury, "torn off")     > 0
replace sev = sev + 1 if strpos(injury, "both")         > 0
replace sev = sev + 1 if strpos(injury, "opened")       > 0
replace sev = sev + 1 if strpos(injury, "cranium")      > 0

replace sev = sev - 2 if strpos(injury, "slight") > 0
replace sev = sev - 1 if strpos(injury, "flesh")  > 0
replace sev = sev - 1 if strpos(injury, "finger") > 0

sum sev
scalar minsev = r(min)
replace sev = sev - minsev
sum sev
scalar maxsev = r(max)
replace sev = sev / maxsev

gen sev2 = sev*sev


(3 real changes made)

(25 real changes made)

(117 real changes made)

(138 real changes made)

(1 real change made)

(10 real changes made)

(15 real changes made)

(1 real change made)

(6 real changes made)

(1 real change made)

(4 real changes made)

(17 real changes made)

(3 real changes made)

(3 real changes made)

(8 real changes made)

(225 real changes made)

(28 real changes made)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         sev |        576    .1354167    1.189401         -3          3

(576 real changes made)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         sev |        576    3.135417    1.189401          0          6

(574 real changes made)



## Final missing check

As a last thing to check, let's just see if the missing observations tend to be more severe. It appears that they arent.

In [12]:
%%stata -s gl

sum sev sev2

sum sev if misdum
sum sev if misdum == 0


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         sev |        576    .5225694    .1982334          0          1
        sev2 |        576    .3123071    .2288797          0          1

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         sev |         78    .5576923    .2060186   .1666667          1

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         sev |        498    .5170683    .1966317          0          1



## Dummies for common regiments

Many of the injuries come from a fixed amount of regiments. Because the regiment determined position on the battlefield, and possibly the distance of the regiment from the battlefield, etc. I should control for the regiment in both the surgical decision and in outcomes. 

In [13]:
%%stata -s gl

gen Ky22  = (regstate == "Kentucky" & regno == "22")
gen Oh16  = (regstate == "Ohio"     & regno == "16")
gen In54  = (regstate == "Indiana"  & regno == "54")
gen Oh42  = (regstate == "Ohio"     & regno == "42")
gen Ia04  = (regstate == "Iowa"     & regno == "4")
gen Il13  = (regstate == "Illinois" & regno == "13")
gen Mo29  = (regstate == "Missouri" & regno == "29")
gen In49  = (regstate == "Indiana"  & regno == "49")
gen Mo06  = (regstate == "Missouri" & regno == "6")
gen Oh114 = (regstate == "Ohio"     & regno == "114")
gen Mo31  = (regstate == "Missouri" & regno == "31")
gen Oh58  = (regstate == "Ohio"     & regno == "58")

# Marker for long names

In an attempt to make use of all possible information in the data, I also use the data on initials. In particular, we note whether or not a soldier has a middle initial. As argued in the paper, this may be correlated (albeit weakly) with social status, and a potential to argue about surgery/amputation. 

In [14]:
%%stata -s gl

replace name = trim(name)
replace name = subinstr(name, ".", "", .)
replace name = subinstr(name, " ", "", .)
replace name = subinstr(name, "Mc", "",.)

gen namelength = length(name)
gen longname   = namelength > 2


(173 real changes made)

(565 real changes made)

(565 real changes made)

(14 real changes made)



## Casualty count by regiment

In the paper, I argue that the frequency with which a regiment occurs in the data could proxy a triage effect, so that a large casaulty count might mean that a wounded soldier, other things equal, should be less likely to get surgery. We use the log of the variable to partially normalize the coefficient. So:

In [15]:
%%stata -s gl

bysort regstate regno: egen casualties = count(regstate)
gen lncas = ln(casualties + 1)

## Relabeling and saving variables

Finally, saving the data set, after dropping missing observations.

In [16]:
%%stata -s gl

drop if misdum

label var sev           "severity"
label var sev2          "severity squared"
label var lncas         "ln(reg. cases)"
label var longname      ">2 initials"
label var operated      "Surgery"
label var Ky22          "22nd Kentucky"
label var Oh16          "16th Ohio"
label var In54          "54th Indiana"
label var Oh42          "42nd Ohio"
label var Ia04          "4th Iowa"
label var Il13          "13th Illinois"
label var Mo29          "29th Missouri"

save "`cdl'\Datafiles\WorkingData.dta", replace

(78 observations deleted)

file C:\Users\mjbaker\Documents\GitHub\CivilWarSurgeryPublic\Datafiles\WorkingData.dta saved



The data should be ready for use in the form that it was applied in in the paper at this point!